In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/contradictory-my-dear-watson/sample_submission.csv
/kaggle/input/contradictory-my-dear-watson/train.csv
/kaggle/input/contradictory-my-dear-watson/test.csv


In [12]:
import pandas as pd
import torch
import torch.nn as nn
import spacy

In [7]:
df_train_dataset = pd.read_csv("../input/contradictory-my-dear-watson/train.csv")
df_test_dataset = pd.read_csv("../input/contradictory-my-dear-watson/test.csv")

In [11]:
df_train_english = df_train_dataset[df_train_dataset["lang_abv"]=="en"]
df_test_english  = df_test_dataset[df_test_dataset["lang_abv"]=="en"]

In [18]:
spacy_eng = spacy.load("en")
df_train_english.head()
print(len(df_train_english))

6870


In [192]:
class LSTM_MODEL(nn.Module):
    def __init__(self,vocab_size,embed_dim,hidden_size,num_classes):
        super(LSTM_MODEL,self).__init__()
        
        self.num_classes = num_classes
        self.embedding = nn.Embedding(vocab_size,embed_dim)
        self.lstm = nn.LSTM(embed_dim,hidden_size,num_layers=5)
        self.Linear = nn.Linear(hidden_size,128)
        self.Linear_2 = nn.Linear(128,128)
        self.Linear_3 = nn.Linear(128,self.num_classes)
        self.dropout = nn.Dropout(0.5)
        
        
    def forward(self,tokenized_text):
        tokenized_premise = tokenized_text[0]
        tokenized_hypothesis = tokenized_text[1]
        premise_embeddings = self.embedding(tokenized_premise)
        print(premise_embeddings.shape)
        premise_hidden , _ = self.lstm(premise_embeddings)  
        premise_outputs = self.Linear(premise_hidden)
        print("DEBUG")

        hypothesis_embeddings = self.dropout(self.embedding(tokenized_hypothesis))
        hypothesis_hidden , _ = self.lstm(hypothesis_embeddings)
        print("HYPOTHESIS HIDDEN SHAPE:",hypothesis_hidden.size())
        hypothesis_outputs = self.Linear(hypothesis_hidden)
        print("DEBUG")
        concat_outputs = torch.cat((premise_outputs,hypothesis_outputs),dim=0)
        fc1 = self.Linear_2(concat_outputs)
        fc_output = self.Linear_3(fc1)

        print(fc_output.shape)
        return fc_output


In [193]:
sample_model_input = [torch.LongTensor([[1,2,4,5],[4,3,2,9]]),torch.LongTensor([[1,2,4,5],[4,3,2,9]])]
lstm_model = LSTM_MODEL(300,300,128,4)

In [195]:
lstm_model(sample_model_input)

torch.Size([2, 4, 300])
DEBUG
HYPOTHESIS HIDDEN SHAPE: torch.Size([2, 4, 128])
DEBUG
torch.Size([4, 4, 4])


tensor([[[-0.1146, -0.0253, -0.0187, -0.0595],
         [-0.1146, -0.0253, -0.0187, -0.0595],
         [-0.1147, -0.0253, -0.0187, -0.0595],
         [-0.1146, -0.0253, -0.0188, -0.0595]],

        [[-0.1148, -0.0263, -0.0189, -0.0596],
         [-0.1148, -0.0263, -0.0190, -0.0597],
         [-0.1148, -0.0263, -0.0189, -0.0596],
         [-0.1148, -0.0264, -0.0190, -0.0596]],

        [[-0.1147, -0.0253, -0.0187, -0.0595],
         [-0.1146, -0.0253, -0.0187, -0.0595],
         [-0.1146, -0.0253, -0.0187, -0.0595],
         [-0.1147, -0.0253, -0.0188, -0.0595]],

        [[-0.1148, -0.0264, -0.0189, -0.0596],
         [-0.1148, -0.0263, -0.0189, -0.0596],
         [-0.1148, -0.0263, -0.0190, -0.0597],
         [-0.1148, -0.0264, -0.0190, -0.0596]]], grad_fn=<AddBackward0>)